# Train

In [1]:
import sys
import random
from contextlib import redirect_stdout

from tensorflow.keras import regularizers, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dense, Dropout, Conv1D, MaxPool1D, Reshape, Concatenate, Flatten, Bidirectional, LSTM,GlobalAveragePooling1D

from dataset import *
from codemaps import *


import sys
from os import system

from tensorflow.keras.models import Model,load_model
import evaluator

In [10]:
#! /usr/bin/python3

def build_network(idx) :

   # sizes
   n_words = codes.get_n_words()
   max_len = codes.maxlen
   n_labels = codes.get_n_labels()

   # word input layer & embeddings
   inpt1 = Input(shape=(max_len,))
   embW1 = Embedding(input_dim=n_words, output_dim=100,input_length=300, mask_zero=False)(inpt1)  

   inpt2 = Input(shape=(max_len,))
   embW2 = Embedding(input_dim=n_words, output_dim=100,input_length=300, mask_zero=False)(inpt2)  

   inpt3 = Input(shape=(max_len,))
   embW3 = Embedding(input_dim=n_words, output_dim=100,input_length=300, mask_zero=False)(inpt3)  

   inpt4 = Input(shape=(max_len,))
   embW4= Embedding(input_dim=n_words, output_dim=100,input_length=300, mask_zero=False)(inpt4)  

   cat = Concatenate(axis=1)([embW1, embW2,embW3,embW4])

   x = Conv1D(filters=30, kernel_size=5, strides=1, activation='relu', padding='same')(cat)
   # x = conv
   # for _ in range(5):
      # x =  Bidirectional(LSTM(units=40, return_sequences=True))(x)

   x= GlobalAveragePooling1D()(x)
   x = Dense(128,activation='relu')(x)
   x = Dense(128,activation='relu')(x)
   out = Dense(n_labels, activation='softmax')(x)

   model = Model([inpt1,inpt2,inpt3,inpt4], out)
   model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

   return model


In [3]:
# directory with files to process
trainfile = '/Users/vicol/Desktop/UPC/ahlt/session6/data/train'
validationfile = '/Users/vicol/Desktop/UPC/ahlt/session6/data/devel'
modelname = 'test'

# load train and validation data
traindata = Dataset(trainfile)
valdata = Dataset(validationfile)

# create indexes from training data
max_len = 150
codes = Codemaps(traindata, max_len)

# encode datasets
Xt = codes.encode_words(traindata)
Yt = codes.encode_labels(traindata)
Xv = codes.encode_words(valdata)
Yv = codes.encode_labels(valdata)




In [5]:
# Xt[0].

(23148, 150)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [12]:
# build network
model = build_network(codes)
with redirect_stdout(sys.stderr) :
   model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 150)]        0           []                               
                                                                                            

In [13]:
# train model
with redirect_stdout(sys.stderr) :
   model.fit(Xt, Yt, batch_size=32, epochs=10, validation_data=(Xv,Yv), verbose=1)
   
# # save model and indexs
# model.save(modelname)
# codes.save(modelname)


Epoch 1/10
120/724 [===>..........................] - ETA: 35s - loss: 0.7523 - accuracy: 0.8383

KeyboardInterrupt: 

# Predict

In [42]:

## --------- Entity extractor ----------- 
## -- Extract drug entities from given text and return them as
## -- a list of dictionaries with keys "offset", "text", and "type"

def output_interactions(data, preds, outfile) :

   #print(testdata[0])
   outf = open(outfile, 'w')
   for exmp,tag in zip(data.sentences(),preds) :
      sid = exmp['sid']
      e1 = exmp['e1']
      e2 = exmp['e2']
      if tag!='null' :
         print(sid, e1, e2, tag, sep="|", file=outf)
            
   outf.close()

   
## --------- MAIN PROGRAM ----------- 
## --
## -- Usage:  baseline-NER.py target-dir
## --
## -- Extracts Drug NE from all XML files in target-dir
## --

# fname = 'test'
datafile = '/Users/vicol/Desktop/UPC/ahlt/session6/data/devel'
outfile = 'stats.out'

# model = load_model(fname)
# codes = Codemaps(fname)

testdata = Dataset(datafile)
X = codes.encode_words(testdata)

Y = model.predict(X)
Y = [codes.idx2label(np.argmax(s)) for s in Y]

# extract relations
output_interactions(testdata, Y, outfile)





145/145 [==============================] - 0s 2ms/step


In [30]:
evaluator.evaluate('DDI','/Users/vicol/Desktop/UPC/ahlt/session6/data/devel','stats.out')

                   tp	  fp	  fn	#pred	#exp	P	R	F1
------------------------------------------------------------------------------
advise             87	  70	  54	 157	 141	55.4%	61.7%	58.4%
effect            143	 111	 169	 254	 312	56.3%	45.8%	50.5%
int                12	   5	  16	  17	  28	70.6%	42.9%	53.3%
mechanism          94	  84	 167	 178	 261	52.8%	36.0%	42.8%
------------------------------------------------------------------------------
M.avg            -	-	-	-	-	58.8%	46.6%	51.3%
------------------------------------------------------------------------------
m.avg             336	 270	 406	 606	 742	55.4%	45.3%	49.9%
m.avg(no class)   383	 223	 359	 606	 742	63.2%	51.6%	56.8%


In [43]:
evaluator.evaluate('DDI','/Users/vicol/Desktop/UPC/ahlt/session6/data/devel','stats.out')

                   tp	  fp	  fn	#pred	#exp	P	R	F1
------------------------------------------------------------------------------
advise             77	  98	  64	 175	 141	44.0%	54.6%	48.7%
effect            175	 223	 137	 398	 312	44.0%	56.1%	49.3%
int                16	   3	  12	  19	  28	84.2%	57.1%	68.1%
mechanism          80	  69	 181	 149	 261	53.7%	30.7%	39.0%
------------------------------------------------------------------------------
M.avg            -	-	-	-	-	56.5%	49.6%	51.3%
------------------------------------------------------------------------------
m.avg             348	 393	 394	 741	 742	47.0%	46.9%	46.9%
m.avg(no class)   409	 332	 333	 741	 742	55.2%	55.1%	55.2%


In [53]:
# return [Xw,Xlw,Xl,Xp]
# 
# model = Model([inptW, inptWLC, inptS, inptP, inptExternalKnowledge], out)